### Download data and install required libraries 

In [1]:
!git clone https://github.com/stasaki/DEcode.git

Cloning into 'DEcode'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 83 (delta 15), reused 49 (delta 5), pack-reused 6
Unpacking objects: 100% (83/83), done.


In [2]:
%cd DEcode

/content/DEcode


In [3]:
!pip install --upgrade setuptools
!pip2 install shap==0.27.0
!Rscript functions/install_package.R

Requirement already up-to-date: setuptools in /usr/local/lib/python2.7/dist-packages (41.2.0)
     |████████████████████████████████| 225kB 48.3MB/s 
  Created wheel for shap: filename=shap-0.27.0-cp27-cp27mu-linux_x86_64.whl size=316528 sha256=70f401d43e1f21bc064a1868424694c48aa3904f50ed167439485ab40d88ac5f
  Stored in directory: /root/.cache/pip/wheels/64/d5/0a/e3b56a63e961ee9afe3c72f0b7cb9bae172c86bd63763c6655
Successfully built shap
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/rjson_0.2.20.tar.gz'
Content type 'application/x-gzip' length 99600 bytes (97 KB)
downloaded 97 KB

* installing *source* package ‘rjson’ ...
** package ‘rjson’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
g++ -std=gnu++11 -I"/usr/share/R/include" -DNDEBUG     -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-uuRxut/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdat

### Set location of input data

In [0]:
# Set input data

# Gene expression matrix
deg_data_file = "./data/toy/Transcriptome/expdata.txt"

# Location of RNA features
mRNA_data_loc = "./data/toy/RNA_features/"
mRNA_annotation_data = ["POSTAR","TargetScan"]

# Location of promoter features
promoter_data_loc = "./data/toy/Promoter_features/"
promoter_annotation_data = ["GTRD"]

# Genes used for traning, validation, and testing
train_genes = "./data/toy/Gene_splits/train.txt.gz"
validate_genes = "./data/toy/Gene_splits/validate.txt.gz"
test_genes = "./data/toy/Gene_splits/test.txt.gz"    

# Location of hyper-parameter
params_loc='./pretrained/Tissue_gene_params.json'

# Output directory
outloc='./train_out/full_model/'

### Define main function

In [0]:
import os
import sys
sys.path.append('./functions/')
import data
import model_utils
import pandas as pd
import numpy as np
import json

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

def main(params):
    from datetime import datetime
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import load_model
    import numpy as np
    import metrics
    import layer_utils
    import network
    
    print(params)
    just_return_model=False
    
    # model parameters and learning parameters
    max_epoch = 100
    batch_size = 128
    
    # batch initialization
    train_steps, train_batches = data.batch_iter(X_mRNA_train.values[:,1],
                                                 X_promoter_train.values[:,1],
                                                 Y_train.values[:,1:],
                                                 batch_size,
                                                 shuffle=True)
    valid_steps, valid_batches = data.batch_iter(X_mRNA_validate.values[:,1],
                                                 X_promoter_validate.values[:,1],
                                                 Y_validate.values[:,1:],
                                                 batch_size,
                                                 shuffle=True)
    test_steps, test_batches = data.batch_iter(X_mRNA_test.values[:,1],
                                               X_promoter_test.values[:,1],
                                               Y_test.values[:,1:],
                                               batch_size,
                                               shuffle=True)

    # Paramters for network structure
    params['n_feature_mRNA']=X_mRNA_train.values[:,1][0].shape[0]
    params['n_feature_promoter']=X_promoter_train.values[:,1][0].shape[0]
    params['n_out'] = Y_train.values[:,1:].shape[1]
    
    # Define network structure
    model = network.define_network(params)
    
    # If you don't need to traning model and just want to have model structure
    if just_return_model:
        return model
    
    # Set callback functions to early stop training and save the best model so far
    time_stamp=datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                    ModelCheckpoint(outloc+time_stamp+'_model.h5', monitor='val_loss', verbose=0,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='min', period=1)]
    
    # Optimizing model
    result = model.fit_generator(train_batches, train_steps, 
                                 epochs=max_epoch,
                                 validation_data=valid_batches,
                                 validation_steps=valid_steps,
                                 callbacks=callbacks,
                                 max_queue_size=10,
                                 verbose=0)
    
    # Test performance
    # Load best model
    model = load_model(outloc+time_stamp+'_model.h5',
                   custom_objects={'pcor': metrics.pcor,
                                  'GlobalSumPooling1D': layer_utils.GlobalSumPooling1D})
    test_performance= np.array(model.evaluate_generator(test_batches,test_steps))
    np.savetxt(outloc+time_stamp+'_test_performance.txt',
               test_performance,delimiter="\t")
    
    # Saving optimization history
    with open(outloc+time_stamp+'_history.json', 'w') as f:
        json.dump(result.history, f)
    
    # Saving model and learning paramters
    with open(outloc+time_stamp+'_params.json', 'w') as f:
        json.dump(params, f)
    
    # Return validation loss for model selection
    validation_loss = np.amin(result.history['val_loss']) 
    
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

### Training model

In [6]:
! mkdir -p "$outloc"
shuffle="None"
# Prepare learning data
Y_train, Y_validate, Y_test, X_mRNA_train, X_mRNA_validate, X_mRNA_test, X_promoter_train, X_promoter_validate, X_promoter_test = data.prep_ml_data_split(
    deg_data_file=deg_data_file,
    mRNA_data_loc=mRNA_data_loc,
    mRNA_annotation_data=mRNA_annotation_data,
    promoter_data_loc=promoter_data_loc,
    promoter_annotation_data=promoter_annotation_data,
    train_genes=train_genes,
    validate_genes=validate_genes,
    test_genes=test_genes,
    outloc=outloc,
    shuffle=shuffle)

# Obtain hyper parameters
with open(params_loc) as f:
    params=json.load(f)  

# Training model
from hyperopt import STATUS_OK
for i in range(10):
    main(params)

# Summarizing the result
! Rscript functions/find_best_model.R "$outloc" &> /dev/null 
with open(outloc+"/summary/best_model.txt") as f:
    best_model=f.readline().rstrip()

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
Using TensorFlow backend.
W0826 19:20:53.835948 140285664106368 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_

{u'n_feature_mRNA': 372, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 726, u'lr': 0.001, u'RNA_n_channel_1st': 160.0, u'Last_n_channel': 160.0, u'RNA_conv_strides': 2.0, u'DNA_n_ConvLayer': 4.0, u'RNA_n_ConvLayer': 1.0, u'n_out': 54, u'Add_basic_info': u'No'}


W0826 19:20:54.049827 140285664106368 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0826 19:20:54.149852 140285664106368 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 19:20:54.603009 140285664106368 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



{u'n_feature_mRNA': 271, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 601, u'lr': 0.001, u'RNA_n_channel_1st': 160.0, u'Last_n_channel': 160.0, u'RNA_conv_strides': 2.0, u'DNA_n_ConvLayer': 4.0, u'RNA_n_ConvLayer': 1.0, u'n_out': 54, u'Add_basic_info': u'No'}
{u'n_feature_mRNA': 271, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 601, u'lr': 0.001, u'RNA_n_channel_1st': 160.0, u'Last_n_channel': 160.0, u'RNA_conv_strides': 2.0, u'DNA_n_ConvLayer': 4.0, u'RNA_n_ConvLayer': 1.0, u'n_out': 54, u'Add_basic_info': u'No'}
{u'n_feature_mRNA': 271, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 601, u'lr': 0.001, u'RN

In [0]:
with open(outloc+"/summary/best_model.txt") as f:
    best_model=f.readline().rstrip()


### Compute Spearman's correlation between actual and predicted expression for each sample

In [0]:
# Prediction for test samples with the best model
model_utils.test_prediction(outloc,
                            best_model,
                            X_mRNA_test,
                            X_promoter_test,
                            Y_test)

In [9]:
! Rscript --vanilla --slave functions/calc_performance.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/test_data/cor_tbl.txt',sep="\t")

,sample,estimate,statistic,p.value,method,alternative
0,Adipose_Subcutaneous,0.207047,2.601248e+06,6.181839e-04,Spearman's rank correlation rho,two.sided
1,Adipose_Visceral_Omentum,0.051037,3.113030e+06,4.035610e-01,Spearman's rank correlation rho,two.sided
2,Adrenal_Gland,-0.007165,3.303958e+06,9.067157e-01,Spearman's rank correlation rho,two.sided
3,Artery_Aorta,0.204438,2.609805e+06,7.263359e-04,Spearman's rank correlation rho,two.sided
4,Artery_Coronary,0.082329,3.010378e+06,1.773940e-01,Spearman's rank correlation rho,two.sided
5,Artery_Tibial,0.304999,2.279920e+06,3.210050e-07,Spearman's rank correlation rho,two.sided
6,Bladder,0.258763,2.431595e+06,1.664295e-05,Spearman's rank correlation rho,two.sided
7,Brain_Amygdala,0.411786,1.929609e+06,1.786476e-12,Spearman's rank correlation rho,two.sided
8,Brain_Anterior_cingulate_cortex_BA24,0.388998,2.004365e+06,3.472624e-11,Spearman's rank correlation rho,two.sided
9,Brain_Caudate_basal_ganglia,0.368323,2.072187e+06,4.237770e-10,Spearman's rank correlation rho,two.sided


### Compute average DeepLIFT score for each regulator

In [10]:
# Estimate variable imporance using test samples
model_utils.compute_DeepLIFT(outloc,
                             best_model,
                             X_mRNA_test,
                             X_promoter_test,
                             Y_test)

0


W0826 19:27:07.657358 140285664106368 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/shap/explainers/deep/deep_tf.py:450: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [11]:
! Rscript --vanilla --slave functions/summarize_DeepLIFT.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/DeepLIFT/RNA_importance_mean.txt',sep="\t")
pd.read_csv(outloc+best_model+'/DeepLIFT/promoter_importance_mean.txt',sep="\t")

,sample_name,AEBP2,AHR,AHRR,APC,AR,ARID1A,ARID1B,ARID2,ARID3A,ARID4B,ARNT,ARNT2,ARNTL,ASCL1,ASCL2,ATF1,ATF2,ATF3,ATF4,ATF5,ATF6,ATF7,ATOH1,ATRX,BACH1,BACH2,BARHL1,BARX1,BARX2,BATF,BATF3,BCL11A,BCL11B,BCL3,BCL6,BHLHE40,BRD4,CDC5L,CDX2,...,ZNF623,ZNF629,ZNF639,ZNF644,ZNF652,ZNF654,ZNF664,ZNF680,ZNF687,ZNF692,ZNF7,ZNF701,ZNF711,ZNF740,ZNF75A,ZNF76,ZNF766,ZNF768,ZNF770,ZNF778,ZNF784,ZNF792,ZNF8,ZNF816,ZNF83,ZNF84,ZNF85,ZNF92,ZSCAN16,ZSCAN2,ZSCAN21,ZSCAN22,ZSCAN29,ZSCAN31,ZSCAN4,ZSCAN5A,ZSCAN5DP,ZSCAN9,ZXDB,ZXDC
0,Adipose_Subcutaneous,3.346075e-04,-0.004190,-0.001368,0.002606,-0.062573,0.000075,0.007642,-0.000335,-0.000493,-0.001375,-0.005749,-1.830860e-04,0.000345,-0.020718,-0.008279,0.004546,0.002894,-0.008350,0.000932,-0.000491,0.000658,-0.000438,0.000031,0.000096,-0.003111,0.001163,-0.000071,0.000517,0.003132,0.005774,-3.432590e-07,-0.001290,-0.004309,0.010666,-0.013139,-0.000942,-0.002903,0.000035,-0.006180,...,-0.000194,0.003130,-0.000694,0.000835,0.000603,2.442221e-04,-0.000174,0.000722,0.002802,-0.001888,-0.001804,-0.002768,0.004027,0.000770,0.001181,-0.005704,-0.000201,0.000035,-0.008549,0.005569,-0.001170,0.003127,0.000216,-0.000632,-0.000071,-0.000059,-0.001268,-0.000075,0.000177,0.000506,0.000527,0.005083,-0.002429,0.000544,0.003949,0.002405,0.001795,-0.001066,0.001416,-0.005460
1,Adipose_Visceral_Omentum,5.258322e-04,-0.005644,-0.001788,0.002077,-0.049528,0.000107,0.007555,-0.000276,-0.001579,-0.002271,-0.001860,-3.674158e-04,0.000844,-0.015808,-0.002019,0.001558,-0.001019,-0.012159,0.000908,-0.000238,0.000994,-0.000678,0.000087,-0.000832,-0.003887,0.001083,0.000188,0.001712,0.002195,0.006735,-2.544933e-04,-0.002472,-0.001576,0.005200,-0.011960,-0.000952,-0.002700,0.000052,-0.001121,...,0.000098,0.000595,-0.002171,0.000691,0.002948,3.053344e-04,0.000147,0.000578,0.002627,-0.002198,-0.002049,-0.003501,0.003370,0.001063,0.001513,-0.004702,-0.000113,0.000230,-0.007775,0.005897,-0.001424,0.005519,0.000307,-0.000712,0.002219,-0.000285,-0.001299,-0.000061,-0.001082,0.000617,0.000572,-0.002927,-0.003070,0.000086,0.002724,0.002127,0.001135,-0.002844,0.000620,-0.004052
2,Adrenal_Gland,1.616744e-04,-0.001091,0.000101,0.000941,-0.015609,0.000336,-0.001759,0.000812,-0.000471,-0.000373,-0.000260,-2.321270e-05,0.000503,0.010186,0.003824,0.003171,-0.001237,0.002315,-0.000263,0.000619,-0.000421,-0.000630,-0.000114,-0.000363,-0.000169,0.001511,0.000112,0.000876,-0.000568,-0.000618,-3.551392e-04,0.001604,0.003532,-0.008504,0.003524,0.002026,0.000428,-0.000067,0.004585,...,0.000255,-0.005726,0.000079,0.000374,0.002368,-1.357475e-04,0.000001,-0.000833,-0.000514,0.000258,0.000381,0.000358,-0.001470,0.000568,-0.000220,0.001746,0.000027,0.000369,-0.000815,-0.000752,0.000020,0.004251,-0.000145,-0.000030,-0.001555,-0.000008,0.000431,-0.000262,-0.000191,-0.000639,-0.000142,-0.004132,-0.000012,0.000029,-0.001197,-0.000962,-0.001244,0.004285,0.001105,0.000742
3,Artery_Aorta,3.575876e-07,-0.000303,-0.001413,0.002678,-0.020910,0.000138,0.004456,-0.000874,0.000052,-0.003106,-0.001607,8.057584e-05,0.000074,-0.006889,-0.007726,-0.000274,0.002400,0.004322,0.000792,-0.000488,0.000508,-0.000214,0.000028,-0.000123,-0.000366,0.000372,-0.000064,-0.000042,0.002343,-0.000068,3.239651e-04,-0.005569,-0.004989,0.010829,-0.002231,0.001946,-0.001602,0.000028,0.000057,...,-0.000316,0.004338,-0.003658,0.002804,-0.001246,-1.374252e-05,-0.000493,0.001356,0.000158,-0.000518,-0.001514,-0.004576,0.004982,-0.000258,0.000939,-0.003750,-0.000151,0.000219,-0.008283,0.002297,-0.000570,0.003007,-0.000067,-0.000522,0.003586,-0.000414,-0.000347,0.000051,0.000828,0.001428,0.000329,0.000340,-0.001405,0.000834,0.001912,0.001723,-0.000524,-0.000886,0.000283,-0.002274
4,Artery_Coronary,-2.994203e-04,0.000762,-0.001242,0.002350,-0.028607,0.001075,0.004310,-0.000891,-0.000326,-0.001023,-0.003893,-9.579547e-05,0.000890,-0.010784,-0.008674,-0.001377,0.002305,-0.003959,0.000547,-0.000381,-0.000296,-0.001948,0.000184,0.000069,-0.000681,0.000796,-0.000068,0.000910,0.001072,0.001799,2.871171

### Simulate the concequence of regulator knockout

In [0]:
# Simulate the concequence of regulator knockout
genes=["ENSG00000268903","ENSG00000239906"]
model_utils.coexpression_with_KO(genes,
                                 outloc,
                                 best_model,
                                 X_mRNA_test,
                                 X_promoter_test,
                                 Y_test)    

In [13]:
! Rscript --vanilla --slave functions/test_KO.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/regulator_KO/regression_res.txt',sep="\t")

,term,estimate,std.error,statistic,p.value
0,EGR1,0.019071,0.000549,34.754318,6.349288e-250
1,ESR1,0.013733,0.000548,25.042667,2.778717e-134
2,WT1,-0.010967,0.000548,-20.001986,2.684291e-87
3,CEBPA,-0.010542,0.000549,-19.219337,7.304835e-81
4,ZNF18,-0.010246,0.000548,-18.683391,1.358906e-76
5,ZBTB33,0.007998,0.000549,14.573703,1.271898e-47
6,MYCN,0.007575,0.000548,13.816717,5.026077e-43
7,ESRRA,0.006425,0.000548,11.715854,1.699357e-31
8,SP1,0.006124,0.000548,11.168537,8.610233e-29
9,ZNF592,0.006016,0.000549,10.959262,8.637257e-28


### Simulate the concequence of binding site removals

In [0]:
# Simulate the concequence of binding site removals
genes=["ENSG00000268903","ENSG00000239906"]
model_utils.coexpression_with_binding_site_removal(genes,
                                                   outloc,
                                                   best_model,
                                                   X_mRNA_test,
                                                   X_promoter_test,
                                                   Y_test)

In [15]:
! Rscript --vanilla --slave functions/test_interval.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/binding_site_removal/regression_res.txt',sep="\t")

,term,estimate,std.error,statistic,p.value,Gene
0,promoter_interval_27,1.404362e-02,2.082604e-04,6.743297e+01,0.000000e+00,ENSG00000268903
1,promoter_interval_28,1.434568e-02,2.083366e-04,6.885817e+01,0.000000e+00,ENSG00000268903
2,promoter_interval_30,6.450572e-04,6.514633e-18,9.901667e+13,0.000000e+00,ENSG00000239906
3,promoter_interval_14,7.401378e-03,2.083081e-04,3.553093e+01,1.834235e-260,ENSG00000268903
4,promoter_interval_11,5.935169e-03,2.082483e-04,2.850044e+01,7.945986e-172,ENSG00000268903
5,promoter_interval_10,-5.225048e-03,2.084236e-04,-2.506937e+01,1.494389e-134,ENSG00000268903
6,promoter_interval_13,4.137921e-03,2.083907e-04,1.985656e+01,4.407078e-86,ENSG00000268903
7,promoter_interval_12,-3.136259e-03,2.082211e-04,-1.506216e+01,1.036337e-50,ENSG00000268903
8,promoter_interval_16,2.590693e-03,2.082527e-04,1.244014e+01,2.889767e-35,ENSG00000268903
9,promoter_interval_15,-2.217819e-03,2.083146e-04,-1.064649e+01,2.507232e-26,ENSG00000268903
